In [ ]:
#!pip install ultralytics


  Using cached ultralytics-8.3.226-py3-none-any.whl.metadata (37 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
Using cached ultralytics-8.3.226-py3-none-any.whl (1.1 MB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 1.5 MB/s eta 0:00:05
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------------------- ---------------

In [9]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
from ultralytics import YOLO
from datetime import datetime
import os

# List of your YAML dataset configs
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"
experiments = [
    ("virtual-only", os.path.join(base_dir, "virtual.yaml"))
]

# Loop through experiments
for name, yaml_path in experiments:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{name}_{timestamp}"
    print(f"\n🚀 Starting training: {name}\n")

    model = YOLO("yolov8n.pt")  # you can change to yolov8s.pt if you want a bit more accuracy
    model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        project="runs_yolo",
        name=run_name,
        seed=42,
        val=True,          # enable validation after each epoch
        exist_ok=True,     # overwrite folder if re-running
    )

print("\n✅ All experiments completed!")



🚀 Starting training: virtual-only

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.226  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data\virtual.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0

RuntimeError: Dataset 'C://Users/giorg/Downloads/drone-detection-distance-template/drone-detection-distance/data/real_LRDD/real.yaml' error  'C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data\real_LRDD\real.yaml' does not exist

In [8]:


# List of your YAML dataset configs
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"
experiments = [
    
    ("real-only", os.path.join(base_dir, "real_LRDD.yaml")),
    ("mixed-50-50", os.path.join(base_dir, "mixed_LRDD.yaml"))
]

# Loop through experiments
for name, yaml_path in experiments:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{name}_{timestamp}"
    print(f"\n🚀 Starting training: {name}\n")

    model = YOLO("yolov8n.pt")  # you can change to yolov8s.pt if you want a bit more accuracy
    model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        project="runs_yolo",
        name=run_name,
        seed=42,
        val=True,          # enable validation after each epoch
        exist_ok=True,     # overwrite folder if re-running
    )

print("\n✅ All experiments completed!")



🚀 Starting training: real-only

Ultralytics 8.3.226  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data\real_LRDD.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=real-only_20251112

## Now we try with the 70/30 datasets in different variations

In [15]:
from ultralytics import YOLO
from datetime import datetime
import os

# === COMMON SETTINGS (fits 4GB GPU) ===
COMMON = dict(
    epochs=50,
    imgsz=640,
    batch=4,          # it was 8
    workers=2,
    seed=42,
    project="runs_yolo",
)

# === PATHS ===
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"

# YAML files
yaml_mixed_70_30 = os.path.join(base_dir, "mixed_LRDD_70_30.yaml")
yaml_virtual = os.path.join(base_dir, "virtual.yaml")
yaml_real = os.path.join(base_dir, "real_LRDD.yaml")

# === EXPERIMENT 1: Baseline 70/30 Mixed ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 1: 70/30 Mixed Baseline ({timestamp})\n")

model = YOLO("yolov8n.pt")
model.train(
    data=yaml_mixed_70_30,
    name=f"mixed_70_30_baseline_{timestamp}",
    **COMMON
)

# === EXPERIMENT 2: Curriculum (Pretrain Virtual -> Finetune Real) ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 2: Curriculum (Virtual → Real) ({timestamp})\n")

# Stage A - Pretrain on virtual (short warm-up)
model = YOLO("yolov8n.pt")
model.train(
    data=yaml_virtual,
    epochs=15,
    imgsz=640,
    batch=4,
    workers=2,
    project="runs_yolo",
    name=f"curriculum_stageA_virtual_{timestamp}",
)
# Stage B - Fine-tune on real
model = YOLO(
    fr"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\notebooks\runs_yolo\curriculum_stageA_virtual_{timestamp}\weights\best.pt"

)

model.train(
    data=yaml_real,
    lr0=0.002,     # smaller LR to preserve learned features
    freeze=10,     # optional: freeze early layers initially
    name=f"curriculum_stageB_real_{timestamp}",
    **COMMON
)

# === EXPERIMENT 3: 70/30 Mixed + Strong Augmentation ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 3: 70/30 Mixed with Strong Augmentation ({timestamp})\n")

model = YOLO("yolov8n.pt")
model.train(
    data=yaml_mixed_70_30,
    name=f"mixed_70_30_augmented_{timestamp}",
    hsv_h=0.03, hsv_s=0.9, hsv_v=0.6,
    erasing=0.6, translate=0.2, scale=0.6, shear=2.0, perspective=0.0005,
    mosaic=0.7, close_mosaic=10,
    **COMMON
)

print("\n All experiments completed successfully!\n")




 Starting Experiment 1: 70/30 Mixed Baseline (20251114_0952)

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.226  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data\mixed_LRDD_70_30.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, 

# break


In [ ]:
# Stage B - Fine-tune on real
model = YOLO(
    fr"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\notebooks\runs_yolo\curriculum_stageA_virtual_{timestamp}\weights\best.pt"

)

model.train(
    data=yaml_real,
    lr0=0.002,     # smaller LR to preserve learned features
    freeze=10,     # optional: freeze early layers initially
    name=f"curriculum_stageB_real_{timestamp}",
    **COMMON
)

# === EXPERIMENT 3: 70/30 Mixed + Strong Augmentation ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"\n Starting Experiment 3: 70/30 Mixed with Strong Augmentation ({timestamp})\n")

model = YOLO("yolov8n.pt")
model.train(
    data=yaml_mixed_70_30,
    name=f"mixed_70_30_augmented_{timestamp}",
    hsv_h=0.03, hsv_s=0.9, hsv_v=0.6,
    erasing=0.6, translate=0.2, scale=0.6, shear=2.0, perspective=0.0005,
    mosaic=0.7, close_mosaic=10,
    **COMMON
)

print("\n All experiments completed successfully!\n")

In [ ]:
# EVALUATION
from glob import glob
import pandas as pd

test_yaml = os.path.join(base_dir, "real_LRDD", "real.yaml")

results = []
for folder in glob("runs_yolo/detect/*/weights/best.pt"):
    name = folder.split(os.sep)[-3]  # extract run name
    print(f"\n🔍 Evaluating {name} on real test set...")
    metrics = YOLO(folder).val(data=test_yaml, split="test", imgsz=512, batch=8)
    results.append({
        "Run": name,
        "mAP50": metrics.results_dict["metrics/mAP50(B)"],
        "mAP50-95": metrics.results_dict["metrics/mAP50-95(B)"],
        "Precision": metrics.results_dict["metrics/precision(B)"],
        "Recall": metrics.results_dict["metrics/recall(B)"],
    })

pd.DataFrame(results).to_csv("runs_yolo/summary_metrics.csv", index=False)
print("\n📊 Summary saved to runs_yolo/summary_metrics.csv")
